In [8]:
import numpy as np
import matplotlib.pyplot as plt

from utils import *

from PIL import Image
import torch
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

In [6]:
data_dir = "../Data_Cropped_and_Resized"

### Transformer feature extraction
https://huggingface.co/DunnBC22/vit-base-patch16-224-in21k_covid_19_ct_scans

In [5]:
extractor = AutoFeatureExtractor.from_pretrained("DunnBC22/vit-base-patch16-224-in21k_covid_19_ct_scans")
model = AutoModelForImageClassification.from_pretrained("DunnBC22/vit-base-patch16-224-in21k_covid_19_ct_scans")
model.eval()

/Users/ktnorton/anaconda3/envs/w281/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [9]:
def transformer_feature_vector(image, model, extractor):
  # Preprocess the image using the feature extractor
  image = Image.fromarray(image).convert('RGB')
  inputs = extractor(images=image, return_tensors="pt")

  # Forward pass through the model's transformer (without the classification head)
  with torch.no_grad():
      feature_vector = model.vit(**inputs).last_hidden_state

  # Optionally, convert to a NumPy array
  feature_vector = feature_vector.numpy()

  return feature_vector

In [11]:
train_path = f"{data_dir}/train"

_, train_transformer_feat, _ = extract_features(train_path, transformer_feature_vector, {"model": model, "extractor": extractor})